<a href="https://colab.research.google.com/github/VijayTaori/Lifo_step1/blob/main/Detailed_Lifo_step4_until_if_statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import calendar
df_open = pd.read_csv('/content/et_open_alldata.csv')
df_close = pd.read_csv('/content/et_close_alldata.csv')
#df_open['open_unix_timestamp'] = pd.to_numeric(df_open['open_unix_timestamp'])
df_close['Close Datetime'] = pd.to_datetime(df_close['Close_unix_timestamp'],dayfirst=True,unit='s')
df_open['Open Datetime'] = pd.to_datetime(df_open['Open_unix_timestamp'],dayfirst=True,unit='s')
df_close_btc = df_close[df_close['Close Action'] =='Buy Bitcoin']
df_open_btc = df_open[df_open['Open Action'] =='Buy Bitcoin']
df_close_btc.reset_index(inplace=True)
df_close_btc.drop('index', axis=1, inplace=True)
df_close_btc.reset_index(inplace=True)
df_close_btc.rename(columns={'index':'Close_index'},inplace=True)
df_open_btc.reset_index(inplace=True)
df_open_btc.drop('index', axis=1, inplace=True)
df_open_btc.reset_index(inplace=True)
df_open_btc.rename(columns={'index':'Open_index'},inplace=True)
data_btc = {}
df_lifo_btc = pd.DataFrame(data_btc, 
                           columns=['Open_index','Close_index',
                                    'Open Position ID', 'Close Position ID',
                                    'Open Units','Close Units','Spread_Eur',
                                    'Open Amount_Eur','Amount_close_Eur',
                                    'Open Datetime', 'Close Datetime',
                                    'Open Rate_Eur', 'Close Rate_Eur',
                                    'Open Action', 'Close Action', 
                                    'Open Amount','Close Rate', 'Open Rate',
                                    'Close Spread', 'Close Profit USD',
                                    'Close_Date','Open_Date',
                                    'Close_unix_timestamp','EurOpen_y', 
                                    'EurHigh_y','EurLow_y', 'EurClose_y',
                                    'EurWeekday_y', 'Open_unix_timestamp',
                                    'EurOpen_x', 'EurHigh_x','EurLow_x', 
                                    'EurClose_x','EurWeekday_x'
                                    ]
                           )
# Let's build a index selector for df_close_btc
close_unit_index = 0
final_close_unit_index = df_open_btc.shape[0]
# for close_unit_index < final_close_unit_index:
value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
close_unit_index = close_unit_index + 0 if value_close_units > 0 else 1
value_close_units
# How to set up an upper limit on this variable??
# Choosing first row from df_close where the units are = values_of_units
# Assign it as new DF
# change the reset the index and the drop the 'index' column
df_latest_row_from_close = df_close_btc[close_unit_index:(close_unit_index+1)]
df_latest_row_from_close.reset_index(inplace=True)
df_latest_row_from_close.drop('index',axis=1,inplace=True)
df_latest_row_from_close
# Get the datetime value from the df_latest_row_from_close
close_unit_datetime = df_close_btc.loc[close_unit_index,'Close Datetime']
close_unit_datetime
# find all the transaction datetimes from df_open that were opened before the close_unit_datetime
open_datetimes_b4 = df_open_btc['Open Datetime'][df_open_btc['Open Datetime'] < close_unit_datetime]
open_datetimes_b4
#Latest datetime beofore close datetime 
open_datetime_lifo = open_datetimes_b4.max()
open_datetime_lifo
# Corresponding row for the open_datetime_lifo 
# let's make a new DF
df_latest_last_row_open = df_open_btc[df_open_btc['Open Datetime'] == open_datetime_lifo]
df_latest_last_row_open.reset_index(inplace=True)
df_latest_last_row_open.drop('index', axis=1,inplace=True)
df_latest_last_row_open
value_open_units = df_latest_last_row_open.loc[0,'Open Units']
value_open_units
value_open_amount = df_latest_last_row_open.loc[0,'Open Amount_Eur']
value_open_amount
value_close_amount = df_latest_row_from_close.loc[0,'Amount_close_Eur']
value_close_amount
value_close_spread_eur = df_latest_row_from_close.loc[0,'Spread_Eur']
value_close_spread_eur
close_Rate_Eur = df_latest_row_from_close.loc[0,'Close Rate_Eur']
close_Rate_Eur
open_Rate_Eur = df_latest_last_row_open.loc[0,'Open Rate_Eur']
open_Rate_Eur
open_index = df_latest_last_row_open.at[0, 'Open_index']
close_index = df_latest_row_from_close.at[0, 'Close_index']
if value_close_units > value_open_units:
  #First lets extract all the values from open DF df_latest_last_row_lifo
  newth_list_open_half = df_latest_last_row_open.to_dict('records')
  newth_dict_open_half = newth_list_open_half[0]
  #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
  df_open_btc.drop(open_index,axis=0,inplace=True)
  #Then reset index and drop the new column 'index' in df_open_btc
  df_open_btc.reset_index(inplace=True)
  df_open_btc.drop('index',axis=1,inplace=True)
  #Now lets work on the Close_side
  #First define the changed values for df_latest_row_from_close
  changed_value_close_units = value_open_units
  changed_value_close_amount = changed_value_close_units * close_Rate_Eur
  changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
  #First change the values of 'units' and 'amount' and 'spread' in the df_latest_row_from_close
  # These values will be reported
  df_latest_row_from_close.at[0,'Close Units'] = changed_value_close_units
  df_latest_row_from_close.at[0,'Amount_close_Eur'] = changed_value_close_amount
  df_latest_row_from_close.at[0,'Spread_Eur'] = changed_value_close_spread
  # Now lets extract all the values from df_latest_row_from_close
  newth_list_close_half = df_latest_row_from_close.to_dict('records')
  newth_dict_close_half = newth_list_close_half[0]
  newth_whole_dict = {**newth_dict_open_half,**newth_dict_close_half}
  # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
  df_lifo_btc = df_lifo_btc.append(newth_whole_dict, ignore_index=True)
  # and after that change the value of units and amount in original df_open_btc
  df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
  df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
  df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
  # Now A
  #df_latest_row_from_close = df_close_btc.loc[close_unit_index, 'Close Units']


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [117]:
df_open_btc[df_open_btc['Open Position ID'] == 611192720]

,Open_index,Open Position ID,Open Action,Open Amount,Open Units,Open Datetime,Open Rate,Open_Date,Open_unix_timestamp,EurOpen_x,EurHigh_x,EurLow_x,EurClose_x,EurWeekday_x,Open Amount_Eur,Open Rate_Eur


In [ ]:
df_close_btc

,Close Position ID,Close Action,Close Units,Close Datetime,Close Rate,Close Spread,Close Profit USD,Close_Date,Close_unix_timestamp,EurOpen_y,EurHigh_y,EurLow_y,EurClose_y,EurWeekday_y,Close Rate_Eur,Amount_close_Eur,Spread_Eur
0,599996081,Buy Bitcoin,0.011288,2020-05-01 11:44:24,9027.00,0.96,27.72,01/05/2020 00:00,1588333464,1.094475,1.101625,1.093542,1.094547,Friday,8194.258482,92.496790,0.700369
1,610187469,Buy Bitcoin,0.058647,2020-05-01 20:55:45,8685.48,3.83,-30.73,01/05/2020 00:00,1588366545,1.094475,1.101625,1.093542,1.094547,Friday,7884.243731,462.387242,3.502380
2,610190271,Buy Bitcoin,0.058881,2020-05-01 20:56:05,8692.19,3.85,-28.20,01/05/2020 00:00,1588366565,1.094475,1.101625,1.093542,1.094547,Friday,7890.334733,464.590799,3.520670
3,609940706,Buy Bitcoin,0.118959,2020-05-01 20:57:04,8699.43,7.79,-25.13,01/05/2020 00:00,1588366624,1.094475,1.101625,1.093542,1.094547,Friday,7896.906842,939.408141,7.123640
4,609941239,Buy Bitcoin,0.112364,2020-05-01 20:57:54,8709.82,7.37,-31.33,01/05/2020 00:00,1588366674,1.094475,1.101625,1.093542,1.094547,Friday,7906.338364,888.387804,6.739567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1368128599,Buy Bitcoin,0.012804,2022-05-12 09:16:23,27867.01,7.21,-425.77,12/05/2022 00:00,1652346983,1.051248,1.052975,1.037355,1.051248,Thursday,26465.025290,338.858184,6.950369
200,1370705010,Buy Bitcoin,0.006407,2022-05-12 09:16:27,27862.74,3.61,-218.86,12/05/2022 00:00,1652346987,1.051248,1.052975,1.037355,1.051248,Thursday,26460.970110,169.535436,3.480004
201,1353186945,Buy Bitcoin,0.001611,2022-05-12 09:16:29,27816.47,0.91,-55.21,12/05/2022 00:00,1652346989,1.051248,1.052975,1.037355,1.051248,Thursday,26417.027940,42.557832,0.877231
202,1361963252,Buy Bitcoin,0.001490,2022-05-12 09:16:32,27799.73,0.84,-52.98,12/05/2022 00:00,1652346992,1.051248,1.052975,1.037355,1.051248,Thursday,26401.130130,39.337684,0.809752


In [108]:
df_latest_last_row_open

,Open_index,Open Position ID,Open Action,Open Amount,Open Units,Open Datetime,Open Rate,Open_Date,Open_unix_timestamp,EurOpen_x,EurHigh_x,EurLow_x,EurClose_x,EurWeekday_x,Open Amount_Eur,Open Rate_Eur
0,54,611192720,Buy Bitcoin,25.0,0.002861,2020-04-30 19:14:44,8736.7,30/04/2020 00:00,1588274084,1.087666,1.096996,1.083565,1.087725,Thursday,23.071989,8062.921929


In [ ]:
#changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
changed_value_close_spread

0.17751206519711638

In [ ]:
changed_value_close_units * value_close_spread_eur

0.0025116182104739995

In [ ]:
value_close_spread_eur

0.877881234

In [ ]:
value_close_amount - changed_value_close_amount

92.496789782998

In [114]:
df_lifo_btc.transpose()

,0
Open_index,32
Close_index,0
Open Position ID,611192720
Close Position ID,599996081
Open Units,0.002861
Close Units,0.002861
Spread_Eur,0.177512
Open Amount_Eur,23.071989
Amount_close_Eur,23.443774
Open Datetime,2020-04-30 19:14:44


In [115]:
df_open_btc

,Open_index,Open Position ID,Open Action,Open Amount,Open Units,Open Datetime,Open Rate,Open_Date,Open_unix_timestamp,EurOpen_x,EurHigh_x,EurLow_x,EurClose_x,EurWeekday_x,Open Amount_Eur,Open Rate_Eur
0,0,599295474,Buy Bitcoin,100.00,0.014029,2020-04-17 13:09:20,7128.14,17/04/2020 00:00,1587128960,1.086012,1.089325,1.081268,1.085847,Friday,92.484010,6592.389676
1,1,599996081,Buy Bitcoin,100.00,0.014149,2020-04-17 18:42:42,7067.56,17/04/2020 00:00,1587148962,1.086012,1.089325,1.081268,1.085847,Friday,92.484010,6536.362863
2,2,600052383,Buy Bitcoin,750.00,0.103664,2020-04-18 13:49:32,7234.89,18/04/2020 00:00,1587217772,1.086012,1.089325,1.081268,1.085847,Saturday,693.630071,6691.116356
3,3,600057583,Buy Bitcoin,150.00,0.020751,2020-04-18 16:48:30,7228.58,18/04/2020 00:00,1587228510,1.086012,1.089325,1.081268,1.085847,Saturday,138.726014,6685.280615
4,4,600061986,Buy Bitcoin,31.81,0.004376,2020-04-18 21:07:10,7269.44,18/04/2020 00:00,1587244030,1.086012,1.089325,1.081268,1.085847,Saturday,29.419163,6723.069581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,199,1353186945,Buy Bitcoin,100.00,0.001611,2021-10-18 21:47:56,62087.07,18/10/2021 00:00,1634593676,1.159595,1.161211,1.157300,1.159703,Monday,86.408019,53648.207030
199,200,1354684971,Buy Bitcoin,438.64,0.006812,2021-10-19 20:54:00,64396.26,19/10/2021 00:00,1634676840,1.161454,1.167000,1.161454,1.161386,Tuesday,377.664548,55444.520400
200,201,1361963252,Buy Bitcoin,94.37,0.001490,2021-10-25 19:00:27,63355.46,25/10/2021 00:00,1635188427,1.164131,1.166861,1.159326,1.164009,Monday,81.400745,54648.528540
201,202,1368128599,Buy Bitcoin,782.56,0.012804,2021-10-29 09:51:32,61119.72,29/10/2021 00:00,1635501092,1.168374,1.169180,1.154668,1.168361,Friday,677.735938,52932.721790


In [ ]:
#First lets extract all the values from open DF df_latest_last_row_lifo
newth_dict_open_half = df_latest_last_row_open.to_dict('records')
type(newth_dict_open_half[0])

dict

In [ ]:
df_open = pd.read_csv('/content/et_open_alldata.csv')
df_close = pd.read_csv('/content/et_close_alldata.csv')
#df_open['open_unix_timestamp'] = pd.to_numeric(df_open['open_unix_timestamp'])

In [ ]:
df_open.columns

In [ ]:
df_close.columns

In [ ]:
#df_open['open_unix_timestamp'] = pd.to_numeric(df_open['open_unix_timestamp'])
df_close['Close Datetime'] = pd.to_datetime(df_close['Close_unix_timestamp'],dayfirst=True,unit='s')
df_open['Open Datetime'] = pd.to_datetime(df_open['Open_unix_timestamp'],dayfirst=True,unit='s')

In [ ]:
df_close

In [ ]:
df_close_btc = df_close[df_close['Close Action'] =='Buy Bitcoin']
df_open_btc = df_open[df_open['Open Action'] =='Buy Bitcoin']
df_close_btc.reset_index(inplace=True)
df_close_btc.drop('index', axis=1, inplace=True)
#df_close_btc.rename(columns={'index':'Close_index'},inplace=True)
df_open_btc.reset_index(inplace=True)
df_open_btc.drop('index', axis=1, inplace=True)
#df_open_btc.rename(columns={'index':'Open_index'},inplace=True)
data_btc = {}

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
data_btc = {}
print(df_close.columns)
df_open.columns

Index(['Close Position ID', 'Close Action', 'Close Units', 'Close Datetime',
       'Close Rate', 'Close Spread', 'Close Profit USD', 'Close_Date',
       'Close_unix_timestamp', 'EurOpen_y', 'EurHigh_y', 'EurLow_y',
       'EurClose_y', 'EurWeekday_y', 'Close Rate_Eur', 'Amount_close_Eur',
       'Spread_Eur'],
      dtype='object')


Index(['Open Position ID', 'Open Action', 'Open Amount', 'Open Units',
       'Open Datetime', 'Open Rate', 'Open_Date', 'Open_unix_timestamp',
       'EurOpen_x', 'EurHigh_x', 'EurLow_x', 'EurClose_x', 'EurWeekday_x',
       'Open Amount_Eur', 'Open Rate_Eur'],
      dtype='object')

In [ ]:
df_lifo_btc = pd.DataFrame(data_btc, 
                           columns=['Open Position ID', 'Close Position ID',
                                    'Open Units','Close Units','Spread_Eur',
                                    'Open Amount_Eur','Amount_close_Eur',
                                    'Open Datetime', 'Close Datetime',
                                    'Open Rate_Eur', 'Close Rate_Eur',
                                    'Open Action', 'Close Action', 
                                    'Open Amount','Close Rate', 'Open Rate',
                                    'Close Spread', 'Close Profit USD',
                                    'Close_Date','Open_Date',
                                    'Close_unix_timestamp','EurOpen_y', 
                                    'EurHigh_y','EurLow_y', 'EurClose_y',
                                    'EurWeekday_y', 'Open_unix_timestamp',
                                    'EurOpen_x', 'EurHigh_x','EurLow_x', 
                                    'EurClose_x','EurWeekday_x'
                                    ]
                           )                      

In [ ]:
'''
df_lifo_btc = pd.DataFrame(data_btc, 
                           columns=['Open Position ID', 'Close Position ID',
                                    'Open Units','Close Units',
                                    'Open Amount_Eur','Amount_close_Eur', 
                                    'Spread_Eur',
                                    'Open Datetime', 'Close Datetime',
                                    'Open Rate_Eur','Close Rate_Eur',
                                    'Open Action','Close Action', 
                                    'Open_unix_timestamp','Close_unix_timestamp',
                                    'Close Spread','Close Profit USD',
                                    'Open Amount'
                                    ]
                           )
'''                           

In [ ]:
df_lifo_btc

,Open Position ID,Close Position ID,Open Units,Close Units,Spread_Eur,Open Amount_Eur,Amount_close_Eur,Open Datetime,Close Datetime,Open Rate_Eur,...,EurHigh_y,EurLow_y,EurClose_y,EurWeekday_y,Open_unix_timestamp,EurOpen_x,EurHigh_x,EurLow_x,EurClose_x,EurWeekday_x


In [ ]:
# Let's build a index selector for df_close_btc
close_unit_index = 0
value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
close_unit_index = close_unit_index + 0 if value_close_units > 0 else 1
value_close_units
# How to set up an upper limit on this variable??

0.014149

In [ ]:
# Choosing first row from df_close where the units are = values_of_units
# Assign it as new DF
# change the reset the index and the drop the 'index' column
df_latest_row_from_close = df_close_btc[close_unit_index:(close_unit_index+1)]
df_latest_row_from_close.reset_index(inplace=True)
df_latest_row_from_close.drop('index',axis=1,inplace=True)
df_latest_row_from_close

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Close Position ID,Close Action,Close Units,Close Datetime,Close Rate,Close Spread,Close Profit USD,Close_Date,Close_unix_timestamp,EurOpen_y,EurHigh_y,EurLow_y,EurClose_y,EurWeekday_y,Close Rate_Eur,Amount_close_Eur,Spread_Eur
0,599996081,Buy Bitcoin,0.014149,2020-05-01 11:44:24,9027.0,0.96,27.72,01/05/2020 00:00,1588333464,1.094475,1.101625,1.093542,1.094547,Friday,8194.258482,115.940563,0.877881


In [ ]:
# Get the datetime value from the df_latest_row_from_close
close_unit_datetime = df_close_btc.loc[close_unit_index,'Close Datetime']
close_unit_datetime

Timestamp('2020-05-01 11:44:24')

In [ ]:
# find all the transaction datetimes from df_open that were opened before the close_unit_datetime
open_datetimes_b4 = df_open_btc['Open Datetime'][df_open_btc['Open Datetime'] < close_unit_datetime]
open_datetimes_b4

In [ ]:
#Latest datetime beofore close datetime 
open_datetime_lifo = open_datetimes_b4.max()
open_datetime_lifo

Timestamp('2020-04-30 19:14:44')

In [ ]:
# Corresponding row for the open_datetime_lifo 
# let's make a new DF
df_latest_last_row_open = df_open_btc[df_open_btc['Open Datetime'] == open_datetime_lifo]
df_latest_last_row_open.reset_index(inplace=True)
df_latest_last_row_open.drop('index', axis=1,inplace=True)
df_latest_last_row_open

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Open Position ID,Open Action,Open Amount,Open Units,Open Datetime,Open Rate,Open_Date,Open_unix_timestamp,EurOpen_x,EurHigh_x,EurLow_x,EurClose_x,EurWeekday_x,Open Amount_Eur,Open Rate_Eur
0,611192720,Buy Bitcoin,25.0,0.002861,2020-04-30 19:14:44,8736.7,30/04/2020 00:00,1588274084,1.087666,1.096996,1.083565,1.087725,Thursday,23.071989,8062.921929


In [ ]:
value_open_units = df_latest_last_row_open.loc[0,'Open Units']
value_open_units

0.002861

In [ ]:
value_open_amount = df_latest_last_row_open.loc[0,'Open Amount_Eur']
value_open_amount

23.07198922

In [ ]:
value_close_units

0.014149

In [ ]:
#df_latest_row_from_close.at[0,'Close Units'] = value_open_units
df_latest_row_from_close.at[0,'Close Units']

In [ ]:
df_latest_row_from_close

,Close Position ID,Close Action,Close Units,Close Datetime,Close Rate,Close Spread,Close Profit USD,Close_Date,Close_unix_timestamp,EurOpen_y,EurHigh_y,EurLow_y,EurClose_y,EurWeekday_y,Close Rate_Eur,Amount_close_Eur,Spread_Eur
0,599996081,Buy Bitcoin,0.014149,2020-05-01 11:44:24,9027.0,0.96,27.72,01/05/2020 00:00,1588333464,1.094475,1.101625,1.093542,1.094547,Friday,8194.258482,115.940563,0.877881


In [ ]:
df_latest_row_from_close.at[0,'Close Units'] = value_open_units
df_latest_row_from_close.at[0,'Amount_close_Eur'] = value_open_amount
df_latest_row_from_close.at[0,'Spread_Eur'] = value_close_spread_eur*

In [ ]:
value_close_amount = df_latest_row_from_close.loc[0,'Amount_close_Eur']
value_close_amount

115.9405633

In [ ]:
value_close_spread_eur = df_latest_row_from_close.loc[0,'Spread_Eur']
value_close_spread_eur

0.877881234

In [ ]:
value_close_spread_eur*value_open_units/value_close_units

0.17751206519711638

In [ ]:
close_Rate_Eur = df_latest_row_from_close.loc[0,'Close Rate_Eur']
close_Rate_Eur

8194.258482

In [ ]:
open_Rate_Eur = df_latest_last_row_open.loc[0,'Open Rate_Eur']
open_Rate_Eur

8062.921929

In [ ]:
if value_close_units > value_open_units:
  #First lets extract all the values from open DF df_latest_last_row_lifo
  newth_dict_open_half = df_latest_last_row_open.to_dict('records')
  #Then permanently delete the first row of the open DF df_open_btc
  df_open_btc.drop(0,axis=0,inplace=True)
  #Then reset index and drop the new column 'index' in df_open_btc
  df_open_btc.reset_index(inplace=True)
  df_open_btc.drop('index',axis=1,inplace=True)
  #Now lets work on the Close_side
  #First define the changed values for df_latest_row_from_close
  changed_value_close_units = value_open_units
  changed_value_close_amount = changed_value_close_units * close_Rate_Eur
  changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
  #First change the values of 'units' and 'amount' and 'spread' in the df_latest_row_from_close
  # These values will be reported
  df_latest_row_from_close.at[0,'Close Units'] = changed_value_close_units
  df_latest_row_from_close.at[0,'Amount_close_Eur'] = changed_value_close_amount
  df_latest_row_from_close.at[0,'Spread_Eur'] = changed_value_close_spread
  # Now lets extract all the values from df_latest_row_from_close
  newth_dict_close_half = df_latest_row_from_close.to_dict('records')
  newth_whole_dict = Merge(newth_dict_open_half,newth_dict_close_half)
  # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
  df_lifo_btc.append(new_row, ignore_index=True)
  # and after that change the value of units and amount in original df_open_btc
  df_open_btc.at[close_unit_index,'Close Units'] = value_open_units - changed_value_close_units
  df_open_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
  df_open_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
  # Now A
  #df_latest_row_from_close = df_close_btc.loc[close_unit_index, 'Close Units']

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


NameError: ignored

In [ ]:
df_latest_last_row_open.loc[0,'Open Units']

0.002861

In [118]:
df_close_btc

,Close_index,Close Position ID,Close Action,Close Units,Close Datetime,Close Rate,Close Spread,Close Profit USD,Close_Date,Close_unix_timestamp,EurOpen_y,EurHigh_y,EurLow_y,EurClose_y,EurWeekday_y,Close Rate_Eur,Amount_close_Eur,Spread_Eur
0,0,599996081,Buy Bitcoin,0.011288,2020-05-01 11:44:24,9027.00,0.96,27.72,01/05/2020 00:00,1588333464,1.094475,1.101625,1.093542,1.094547,Friday,8194.258482,92.496790,0.700369
1,1,610187469,Buy Bitcoin,0.058647,2020-05-01 20:55:45,8685.48,3.83,-30.73,01/05/2020 00:00,1588366545,1.094475,1.101625,1.093542,1.094547,Friday,7884.243731,462.387242,3.502380
2,2,610190271,Buy Bitcoin,0.058881,2020-05-01 20:56:05,8692.19,3.85,-28.20,01/05/2020 00:00,1588366565,1.094475,1.101625,1.093542,1.094547,Friday,7890.334733,464.590799,3.520670
3,3,609940706,Buy Bitcoin,0.118959,2020-05-01 20:57:04,8699.43,7.79,-25.13,01/05/2020 00:00,1588366624,1.094475,1.101625,1.093542,1.094547,Friday,7896.906842,939.408141,7.123640
4,4,609941239,Buy Bitcoin,0.112364,2020-05-01 20:57:54,8709.82,7.37,-31.33,01/05/2020 00:00,1588366674,1.094475,1.101625,1.093542,1.094547,Friday,7906.338364,888.387804,6.739567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,199,1368128599,Buy Bitcoin,0.012804,2022-05-12 09:16:23,27867.01,7.21,-425.77,12/05/2022 00:00,1652346983,1.051248,1.052975,1.037355,1.051248,Thursday,26465.025290,338.858184,6.950369
200,200,1370705010,Buy Bitcoin,0.006407,2022-05-12 09:16:27,27862.74,3.61,-218.86,12/05/2022 00:00,1652346987,1.051248,1.052975,1.037355,1.051248,Thursday,26460.970110,169.535436,3.480004
201,201,1353186945,Buy Bitcoin,0.001611,2022-05-12 09:16:29,27816.47,0.91,-55.21,12/05/2022 00:00,1652346989,1.051248,1.052975,1.037355,1.051248,Thursday,26417.027940,42.557832,0.877231
202,202,1361963252,Buy Bitcoin,0.001490,2022-05-12 09:16:32,27799.73,0.84,-52.98,12/05/2022 00:00,1652346992,1.051248,1.052975,1.037355,1.051248,Thursday,26401.130130,39.337684,0.809752


In [119]:
df_open_btc.shape

(203, 16)